In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing Data

In [ ]:
eda=pd.read_csv('/kaggle/input/titanic-dataset-from-kaggle/train.csv')

# Importing all the Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
eda

# EDA(Exploratory Data Analysis) or Cleaning of the Data

In [ ]:
eda.head()

In [ ]:
eda.tail()

In [ ]:
eda.info()

In [ ]:
eda.describe()

## Checking for Null Values

In [ ]:
eda.isnull().sum()

### We should 1st check whether Null values are present in the DataSet or not, before performing any cleaning on that particular column. Here age has null values,so we should fix it.

In [ ]:
eda['Age']=eda['Age'].fillna(value=eda['Age'].mean())

In [ ]:
eda.isnull().sum()

## Checking for Outliers in the DataSet

In [ ]:
eda.boxplot()

### Outliers are present in four different columns,two of those columns are ignorable,'Age','Fare' Column Outliers should be Fixed.

### It is finded and capped as below:

In [ ]:
IQR_Fare=eda['Fare'].quantile(0.75)-eda['Fare'].quantile(0.25)
IQR_Fare

In [ ]:
Upper_OutlierLimit=eda['Fare'].quantile(0.75)+1.5*IQR_Fare
Upper_OutlierLimit

In [ ]:
OutlierValues=eda[(eda['Fare']>Upper_OutlierLimit)]
OutlierValues

In [ ]:
eda['Fare']=np.where(eda['Fare']>65.6,eda['Fare'].quantile(0.85),eda['Fare'])
eda

In [ ]:
IQR_Age=eda['Age'].quantile(0.75)-eda['Age'].quantile(0.25)
IQR_Age

In [ ]:
Upper_OutlierLimit2=eda['Age'].quantile(0.75)+1.5*IQR_Age
Upper_OutlierLimit2

In [ ]:
OutlierValues2=eda[(eda['Age']>Upper_OutlierLimit2)]
OutlierValues2

In [ ]:
eda['Age']=np.where(eda['Age']>54.5,eda['Age'].quantile(0.95),eda['Age'])
eda

### Now Outliers are capped.We can check it through BoxPlot of the Particular Columns 

In [ ]:
eda.boxplot(column=['Fare'])

In [ ]:
obj=eda.dtypes==np.object
print(obj)

### Here,we have Name,Sex,Ticket,Cabin & Embarked columns as categorical data.
### In the process of Cleaning,we don't require/accept categorical data.All the data should be numeric.
### Other thing to note here is,'Sex' and 'Embarked' Columns are not Continuous data.So we can get numeric data by creating dummies to them.
### So we should Drop 3 Columns and create dummies for 2 columns 

In [ ]:
eda.drop(['Name','Ticket','Cabin'],axis=1,inplace=True)
eda

In [ ]:
eda=pd.get_dummies(eda,drop_first=True)
eda

In [ ]:
cols=eda.columns
cols=['PassengerId','Pclass','Age','SibSp','Parch','Fare','Sex_male','Embarked_Q','Embarked_S','Survived']

In [ ]:
eda=eda[cols]
eda

### We are just interchanging the columns(Dependent variable to last column) just to have good view

# Building the model by using Logistic Regression

### Defining x and y:

In [ ]:
x=eda.iloc[:,:-1].values
x.shape

In [ ]:
y=eda.iloc[:,-1].values
y.shape

### Splitting the DataSet

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25 , random_state=5)

In [ ]:
x_train.shape

In [ ]:
x_test.shape

In [ ]:
y_train.shape

In [ ]:
y_test.shape

### Building the Model

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(x_train, y_train)

In [ ]:
y_pred = lr.predict(x_test)

In [ ]:
y_test

In [ ]:
y_pred

## Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
confusion = confusion_matrix(y_test, y_pred)
print(confusion)

In [ ]:
TN = confusion [0,0]
FP = confusion [0,1]
FN = confusion [1,0]
TP = confusion [1,1]

In [ ]:
print(confusion)
print ("TN: ", TN)
print ("FP: ", FP)
print ("FN: ", FN)
print ("TP: ", TP)

## Classification Accuracy

In [ ]:
from sklearn import metrics
accuracy = metrics.accuracy_score(y_test, y_pred)
accuracy1 = (TN+TP)/(TN+TP+FN+FP)
print ("Accuracy from metrics: ", accuracy)
print ("Accuracy Calculated: ", accuracy1)

## Classification Error

In [ ]:
print((FP+FN)/float(TP+TN+FP+FN))
print(round(1-metrics.accuracy_score(y_test, y_pred),4))

## Sensitivity/True Positive Rate/Recall

In [ ]:
print("RECALL:", metrics.recall_score(y_test,y_pred))
print("CALCULATED RECALL:", (TP)/(TP+FN))

## Specificity/True Negative Rate

In [ ]:
print ("SPECIFICITY/TRUE NEGATIVE RATE:", (TN)/(TN+FP))

## False Positive Rate

In [ ]:
print("FALSE POSITIVE RATE: ",(FN)/(FN+TP))

## False Negative Rate

In [ ]:
print("FALSE NEGATIVE RATE: ",(FP)/(TN+FP))

## Precision

In [ ]:
print ("Precision: ", round(metrics.precision_score(y_test,y_pred),2))
print ("PRECISION CALCULATED: ", round(TP/float(TP+FP),2))

## f1 Score

In [ ]:
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score,roc_auc_score
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

print('Accuracy is  :' ,round(accuracy,2)*100)
print('F1 score is :' ,round(f1,2)*100)
print('Precision is  :',round(precision,2)*100)
print('Recall is  :',round(recall,4)*100)
print('Roc Auc is  :',round(roc_auc,2)*100)